In [1]:
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("8_LDA_GaussianMixture_BisectingKMeans").getOrCreate()
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType, DoubleType

schema = (StructType()
            .add("InvoiceNo", StringType(), True)
            .add("StockCode", StringType(), True)
            .add("Description", StringType(), True)
            .add("Quantity", StringType(), True)
            .add("InvoiceDate", StringType(), True)
            .add("UnitPrice", StringType(), True)
            .add("CustomerID", StringType(), True)
            .add("Country", StringType(), True)
         )

df1 = (spark.read
               .option("header", True)
             .csv("7_data/online_retail.csv"))
df2 = (spark.read
               .option("header", True)
             .csv("7_data/online_retail_II.csv"))
df1.printSchema()
df2.printSchema()

22/10/22 18:44:13 WARN Utils: Your hostname, m0 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/22 18:44:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/22 18:44:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/22 18:44:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Country: string (nullable = true)



In [2]:
retail_df = df1.union(df2)
retail_df.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-01-12|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-01-12|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-01-12|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6| 2010-01-12|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6| 2010-01-12|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2| 2010-01-12|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6| 2010-01-12|     4.25|     17850|United Kingdom|


In [3]:
df3 = (spark.read
             .option("header", True)
             .option("delimiter", ";")
            .csv("7_data/countries_codes.csv"))

country_df = (df3
   .withColumnRenamed("OFFICIAL LANG CODE", "CountryCode")
   .withColumnRenamed("ISO2 CODE", "ISO2Code")
   .withColumnRenamed("ISO3 CODE", "ISO3Code")
   .withColumnRenamed("LABEL EN", "CountryName")
   .withColumnRenamed("Geo Shape", "GeoShape")
   .drop("ONU CODE")
   .drop("IS ILOMEMBER")
   .drop("IS RECEIVING QUEST")
   .drop("LABEL FR")
   .drop("LABEL SP")
   .drop("geo_point_2d")
)

integrated_df = retail_df.join(country_df, retail_df.Country == country_df.CountryName, "left_outer")

In [4]:
country_df.show()
integrated_df.show()

+-----------+--------+--------+--------------------+--------------------+
|CountryCode|ISO2Code|ISO3Code|         CountryName|            GeoShape|
+-----------+--------+--------+--------------------+--------------------+
|         EN|      CY|     CYP|              Cyprus|"{""type"": ""Mul...|
|         EN|      E1|     ET1|Ethiopia (includi...|                null|
|         EN|      SS|     SSD|South Sudan, The ...|                null|
|         EN|      TR|     TUR|              Turkey|"{""type"": ""Mul...|
|         EN|      CN|     CHN|               China|"{""type"": ""Mul...|
|         EN|      LY|     LBY|Libyan Arab Jamah...|"{""type"": ""Pol...|
|         FR|      GW|     GNB|       Guinea-Bissau|"{""type"": ""Mul...|
|         EN|      MW|     MWI|              Malawi|"{""type"": ""Mul...|
|         EN|      BH|     BHR|             Bahrain|"{""type"": ""Pol...|
|         SP|      PE|     PER|                Peru|"{""type"": ""Mul...|
|         EN|      PH|     PHL|       

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----------+--------+--------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|CountryCode|ISO2Code|ISO3Code|   CountryName|            GeoShape|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----------+--------+--------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-01-12|     2.55|     17850|United Kingdom|         EN|      GB|     GBR|United Kingdom|"{""type"": ""Mul...|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-01-12|     3.39|     17850|United Kingdom|         EN|      GB|     GBR|United Kingdom|"{""type"": ""Mul...|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-01-12|     2.75|     17850|United Kingdom|         EN|      GB|     GBR|United Kingdom|"{""type"": ""Mul...|
|   

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType, DoubleType

schema = (StructType()
            .add("age", StringType(), True)
            .add("workclass", StringType(), True)
            .add("fnlwgt", StringType(), True)
            .add("education", StringType(), True)
            .add("education-num", StringType(), True)
            .add("marital-status", StringType(), True)
            .add("occupation", StringType(), True)
            .add("relationship", StringType(), True)
            .add("race", StringType(), True)
            .add("gender", StringType(), True)
            .add("capital-gain", StringType(), True)
            .add("capital-loss", StringType(), True)
            .add("hours-per-week", StringType(), True)
            .add("native-country", StringType(), True)
         )

In [6]:
from pyspark.sql.functions import monotonically_increasing_id

income_df = spark.read.schema(schema).csv("7_data/adult.data").withColumn("idx", monotonically_increasing_id())

retail_dfx = integrated_df.withColumn("RetailIDx", monotonically_increasing_id())
income_dfx = income_df.withColumn("IncomeIDx", monotonically_increasing_id()) 
retail_enriched_df = retail_dfx.join(income_dfx, retail_dfx.RetailIDx == income_dfx.IncomeIDx, "left_outer")

display(retail_enriched_df)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string, CountryCode: string, ISO2Code: string, ISO3Code: string, CountryName: string, GeoShape: string, RetailIDx: bigint, age: string, workclass: string, fnlwgt: string, education: string, education-num: string, marital-status: string, occupation: string, relationship: string, race: string, gender: string, capital-gain: string, capital-loss: string, hours-per-week: string, native-country: string, idx: bigint, IncomeIDx: bigint]

In [7]:
retail_enriched_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- ISO2Code: string (nullable = true)
 |-- ISO3Code: string (nullable = true)
 |-- CountryName: string (nullable = true)
 |-- GeoShape: string (nullable = true)
 |-- RetailIDx: long (nullable = false)
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: string (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-

In [8]:
 retail_clean_df = (retail_enriched_df
                    .drop("Country")
                    .drop("ISO2Code")
                    .drop("ISO3Code")
                    .drop("RetailIDx")
                    .drop("idx")
                    .drop("IncomeIDx")
                   )

In [9]:
(retail_clean_df
   .select("InvoiceNo", "InvoiceDate", "StockCode")
   .groupBy("InvoiceNo", "InvoiceDate", "StockCode")
   .count()
   .show()
)

+---------+-----------+---------+-----+
|InvoiceNo|InvoiceDate|StockCode|count|
+---------+-----------+---------+-----+
|   536381| 2010-01-12|    22438|    1|
|   536401| 2010-01-12|    22549|    1|
|   536406| 2010-01-12|    71053|    1|
|   536408| 2010-01-12|    22716|    1|
|   536415| 2010-01-12|    22739|    1|
|   536415| 2010-01-12|    22565|    1|
|   536423| 2010-01-12|    22619|    1|
|   536425| 2010-01-12|    22768|    1|
|   536525| 2010-01-12|    21791|    1|
|   536531| 2010-01-12|    22193|    1|
|   536536| 2010-01-12|    72817|    1|
|   536544| 2010-01-12|    22348|    1|
|   536544| 2010-01-12|    22517|    1|
|   536569| 2010-01-12|    22945|    1|
|   536569| 2010-01-12|    22639|    1|
|   536569| 2010-01-12|    21823|    1|
|   536592| 2010-01-12|    22195|    1|
|   536592| 2010-01-12|    22536|    1|
|   536592| 2010-01-12|    22742|    1|
|   536592| 2010-01-12|    22747|    1|
+---------+-----------+---------+-----+
only showing top 20 rows



In [10]:
(retail_clean_df
   .select("*")
   .where("InvoiceNo in ('536373', '536382', '536387') AND StockCode in ('85123A', '22798', '21731')")
   .show()
)

+---------+---------+--------------------+--------+-----------+---------+----------+-----------+--------------+--------------------+---+---------+-------+-------------+-------------+-------------------+---------------+--------------+------+------+------------+------------+--------------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|CountryCode|   CountryName|            GeoShape|age|workclass| fnlwgt|    education|education-num|     marital-status|     occupation|  relationship|  race|gender|capital-gain|capital-loss|hours-per-week|native-country|
+---------+---------+--------------------+--------+-----------+---------+----------+-----------+--------------+--------------------+---+---------+-------+-------------+-------------+-------------------+---------------+--------------+------+------+------------+------------+--------------+--------------+
|   536373|   85123A|WHITE HANGING HEA...|       6| 2010-01-12|     2.55|     17850|    

In [11]:
full_df = retail_clean_df.drop_duplicates(["InvoiceNo", "InvoiceDate", "StockCode"])
full_df.show()

22/10/22 18:45:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------+--------------------+--------+-----------+---------+----------+-----------+--------------+--------------------+----+---------+------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|CountryCode|   CountryName|            GeoShape| age|workclass|fnlwgt|education|education-num|marital-status|occupation|relationship|race|gender|capital-gain|capital-loss|hours-per-week|native-country|
+---------+---------+--------------------+--------+-----------+---------+----------+-----------+--------------+--------------------+----+---------+------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+
|   489434|    21232|STRAWBERRY CERAMI...|      24| 2009-01-12|     1.25|     13085|         EN|United Kingdom|"{""type"": ""Mul...|null|     

In [12]:
retail_final_df = (full_df.selectExpr(
                                 "InvoiceNo AS invoice_num", "StockCode AS stock_code", "description AS description",
                                    "Quantity AS quantity", "InvoiceDate AS invoice_date", 
                                    "CAST(UnitPrice AS DOUBLE) AS unit_price", "CustomerID AS customer_id",
                                    "CountryCode AS country_code", "CountryName AS country_name", "GeoShape AS geo_shape",
                                    "age", "workclass AS work_class", "fnlwgt AS final_weight", "education", 
                                    "'education-num' AS education_num", "'marital-status' AS marital_status", 
                                    "occupation", "relationship", "race", "gender", "'capital-gain' AS capital_gain",
                                    "'capital-loss' AS capital_loss", "'hours-per-week' AS hours_per_week",
                                    "'native-country' AS native_country")
                  )
retail_final_df.show()

+-----------+----------+--------------------+--------+------------+----------+-----------+------------+--------------+--------------------+----+----------+------------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+
|invoice_num|stock_code|         description|quantity|invoice_date|unit_price|customer_id|country_code|  country_name|           geo_shape| age|work_class|final_weight|education|education_num|marital_status|occupation|relationship|race|gender|capital_gain|capital_loss|hours_per_week|native_country|
+-----------+----------+--------------------+--------+------------+----------+-----------+------------+--------------+--------------------+----+----------+------------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+
|     489434|     21232|STRAWBERRY CERAMI...|      24|  2009-01-12|      1.25|      13085|          

In [13]:
# retail_final_df.limit(5000).write.format("csv").mode("overwrite")\
# .option("header", "true")\
# .save("7_data/retail_silver.csv") 

# retail_final_df.write.mode("overwrite").csv("7_data/retail_silver.csv") 

# retail_final_df.repartition(1).write.mode("overwrite").csv("7_data/retail_silver.csv") 

# retail_final_df.coalesce(1).write.option("header",True).mode("overwrite").csv("7_data/retail_silver.csv") 

# df = retail_final_df.toPandas()
# df.to_csv("7_data/retail_silver.csv", sep='\t', encoding='utf-8')

# Data wrangling: Data preprocessing

In [14]:
select_data = retail_final_df.select("invoice_num", "stock_code",
 "quantity", "invoice_date",
 "unit_price","country_code","description",
 "age", "work_class",
 "final_weight")
select_data.describe().show()

+-------+------------------+------------------+-----------------+------------+-----------------+------------+--------------------+------------------+------------+------------------+
|summary|       invoice_num|        stock_code|         quantity|invoice_date|       unit_price|country_code|         description|               age|  work_class|      final_weight|
+-------+------------------+------------------+-----------------+------------+-----------------+------------+--------------------+------------------+------------+------------------+
|  count|             34069|             34069|            34069|       34069|            34069|       33782|               33975|             19425|       19425|             19425|
|   mean| 516939.2320726623| 30344.19952786616|8.653937597229152|        null|6.202600898177344|        null|             21494.0|38.600051480051484|        null|189840.43212355213|
| stddev|23351.991183892427|20571.417801166674|80.34506857031046|        null|155.37450893

# Data cleansing

In [15]:
from pyspark.sql.functions import*
from pyspark.sql.types import *

interim_data = (select_data
 .withColumn("invoice_time", col("invoice_date").cast(DateType()))              
 .withColumn("cust_age", col("age").cast(FloatType()))
 .withColumn("working_class", col("work_class").cast(FloatType()))
 .withColumn("fin_wt", col("final_weight").cast(FloatType()))
)
clean_data = interim_data.na.fill(0)

# Data manipulation

In [16]:
clean_data.show(5,False)

+-----------+----------+--------+------------+----------+------------+----------------------------------+----+----------+------------+------------+--------+-------------+------+
|invoice_num|stock_code|quantity|invoice_date|unit_price|country_code|description                       |age |work_class|final_weight|invoice_time|cust_age|working_class|fin_wt|
+-----------+----------+--------+------------+----------+------------+----------------------------------+----+----------+------------+------------+--------+-------------+------+
|489434     |21232     |24      |2009-01-12  |1.25      |EN          |STRAWBERRY CERAMIC TRINKET BOX    |null|null      |null        |2009-01-12  |0.0     |0.0          |0.0   |
|489434     |21523     |10      |2009-01-12  |5.95      |EN          |FANCY FONT HOME SWEET HOME DOORMAT|null|null      |null        |2009-01-12  |0.0     |0.0          |0.0   |
|489434     |21871     |24      |2009-01-12  |1.25      |EN          |SAVE THE PLANET MUG               |null|

In [17]:
int_vars=['quantity']
for column in int_vars:
  clean_data=clean_data.withColumn(column,clean_data[column].cast(IntegerType()))

date_vars=['invoice_date']
#Converting variables
for column in date_vars:
 clean_data=clean_data.withColumn(column,clean_data[column].cast(DateType()))

In [18]:
clean_data.dtypes

[('invoice_num', 'string'),
 ('stock_code', 'string'),
 ('quantity', 'int'),
 ('invoice_date', 'date'),
 ('unit_price', 'double'),
 ('country_code', 'string'),
 ('description', 'string'),
 ('age', 'string'),
 ('work_class', 'string'),
 ('final_weight', 'string'),
 ('invoice_time', 'date'),
 ('cust_age', 'float'),
 ('working_class', 'float'),
 ('fin_wt', 'float')]

# Feature extraction

In [19]:
from pyspark.sql.functions import split, trim
from pyspark.ml.feature import CountVectorizer
cv_df = clean_data.withColumn("desc_array",split(trim("description"), " ")).where("description is NOT NULL")
cv = CountVectorizer(inputCol="desc_array",
                     outputCol="description_vec",
                     vocabSize=2, minDF=2.0)
cv_model = cv.fit(cv_df)
train_df = cv_model.transform(cv_df)
train_df.show()

+-----------+----------+--------+------------+----------+------------+--------------------+----+----------+------------+------------+--------+-------------+------+--------------------+---------------+
|invoice_num|stock_code|quantity|invoice_date|unit_price|country_code|         description| age|work_class|final_weight|invoice_time|cust_age|working_class|fin_wt|          desc_array|description_vec|
+-----------+----------+--------+------------+----------+------------+--------------------+----+----------+------------+------------+--------+-------------+------+--------------------+---------------+
|     489434|     21232|      24|  2009-01-12|      1.25|          EN|STRAWBERRY CERAMI...|null|      null|        null|  2009-01-12|     0.0|          0.0|   0.0|[STRAWBERRY, CERA...|      (2,[],[])|
|     489434|     21523|      10|  2009-01-12|      5.95|          EN|FANCY FONT HOME S...|null|      null|        null|  2009-01-12|     0.0|          0.0|   0.0|[FANCY, FONT, HOM...|      (2,[],

In [20]:
from pyspark.ml.feature import Word2Vec
w2v_df = clean_data.withColumn("desc_array",
split(trim("description"), "\t")).where("description is NOT NULL")
word2vec = Word2Vec(vectorSize=2, minCount=0,
                    inputCol="desc_array", outputCol="desc_vec")
w2v_model = word2vec.fit(w2v_df)
train_df = w2v_model.transform(w2v_df)

# The tokenization of text into individual terms

In [21]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="description",
                      outputCol="desc_terms")
tokenized_df = tokenizer.transform(clean_data)
tokenized_df.show(10, False)

+-----------+----------+--------+------------+----------+------------+-----------------------------------+----+----------+------------+------------+--------+-------------+------+------------------------------------------+
|invoice_num|stock_code|quantity|invoice_date|unit_price|country_code|description                        |age |work_class|final_weight|invoice_time|cust_age|working_class|fin_wt|desc_terms                                |
+-----------+----------+--------+------------+----------+------------+-----------------------------------+----+----------+------------+------------+--------+-------------+------+------------------------------------------+
|489434     |21232     |24      |2009-01-12  |1.25      |EN          |STRAWBERRY CERAMIC TRINKET BOX     |null|null      |null        |2009-01-12  |0.0     |0.0          |0.0   |[strawberry, ceramic, trinket, box]       |
|489434     |21523     |10      |2009-01-12  |5.95      |EN          |FANCY FONT HOME SWEET HOME DOORMAT |null|n

# Removing common words using StopWordsRemover

In [22]:
from pyspark.ml.feature import StopWordsRemover
stops_remover = StopWordsRemover(inputCol="desc_terms",
                                 outputCol="desc_nostops")
stops_df = stops_remover.transform(tokenized_df)
stops_df.select("desc_terms", "desc_nostops").show(10,False)

+------------------------------------------+------------------------------------------+
|desc_terms                                |desc_nostops                              |
+------------------------------------------+------------------------------------------+
|[strawberry, ceramic, trinket, box]       |[strawberry, ceramic, trinket, box]       |
|[fancy, font, home, sweet, home, doormat] |[fancy, font, home, sweet, home, doormat] |
|[save, the, planet, mug]                  |[save, planet, mug]                       |
|["record, frame, 7"", single, size, "]    |["record, frame, 7"", single, size, "]    |
|[pink, doughnut, trinket, pot]            |[pink, doughnut, trinket, pot]            |
|[pink, cherry, lights]                    |[pink, cherry, lights]                    |
|[, white, cherry, lights]                 |[, white, cherry, lights]                 |
|[15cm, christmas, glass, ball, 20, lights]|[15cm, christmas, glass, ball, 20, lights]|
|[heart, measuring, spoons, larg

# Encoding string variables using StringIndexer

In [23]:
from pyspark.ml.feature import StringIndexer
string_indexer = StringIndexer(inputCol="country_code",
                               outputCol="country_indexed",
                               handleInvalid="skip" )
indexed_df = string_indexer.fit(stops_df).transform(stops_df)
indexed_df.select("country_code", "country_indexed").show(10000)

+------------+---------------+
|country_code|country_indexed|
+------------+---------------+
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|          EN|            0.0|
|       

# Transforming a categorical variable into a vector using OneHotEncoder

In [24]:
from pyspark.ml.feature import OneHotEncoder
ohe = OneHotEncoder(inputCol="country_indexed",
                    outputCol="country_ohe")
ohe_df = ohe.fit(indexed_df).transform(indexed_df)
ohe_df.select("country_code", "country_ohe").show()

+------------+-------------+
|country_code|  country_ohe|
+------------+-------------+
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
|          EN|(2,[0],[1.0])|
+------------+-------------+
only showing top 20 rows



# Transforming continuous variables

In [25]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=10, inputCol="unit_price",
                      outputCol="binarized_price")
binarized_df = binarizer.transform(ohe_df)
binarized_df.select("quantity", "binarized_price").show(1000)

+--------+---------------+
|quantity|binarized_price|
+--------+---------------+
|      24|            0.0|
|      10|            0.0|
|      24|            0.0|
|      48|            0.0|
|      24|            0.0|
|      12|            0.0|
|      12|            0.0|
|      12|            0.0|
|      24|            0.0|
|      12|            0.0|
|      12|            0.0|
|      12|            0.0|
|      12|            0.0|
|       6|            0.0|
|       3|            0.0|
|      18|            0.0|
|       3|            0.0|
|       4|            0.0|
|      16|            0.0|
|      24|            0.0|
|       3|            0.0|
|      12|            0.0|
|       2|            0.0|
|      12|            0.0|
|      12|            0.0|
|      12|            0.0|
|      10|            0.0|
|      12|            0.0|
|       8|            0.0|
|       8|            0.0|
|      16|            0.0|
|      12|            0.0|
|       3|            0.0|
|       3|            0.0|
|

# Transforming the date and time variables

In [26]:

from pyspark.sql.functions import month
month_df = binarized_df.withColumn("invoice_month",
                                   month("invoice_date"))



month_indexer = StringIndexer(inputCol="invoice_month",
                              outputCol="month_indexed",
                              handleInvalid="skip" )
month_df = month_indexer.fit(month_df).transform(month_df)
month_df.select("invoice_month", "month_indexed").show(10000)

+-------------+-------------+
|invoice_month|month_indexed|
+-------------+-------------+
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|            1|          0.0|
|         

# Assembling individual features into a feature vector

In [27]:
from pyspark.ml.feature import VectorAssembler
vec_assembler = VectorAssembler(
     inputCols=[ "country_ohe", "binarized_price", "month_indexed"], outputCol="features")
features_df = vec_assembler.transform(month_df)
features_df.select("features").show()

+-------------+
|     features|
+-------------+
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
|(4,[0],[1.0])|
+-------------+
only showing top 20 rows



# Feature scaling

In [28]:
from pyspark.ml.feature import StandardScaler
std_scaler = StandardScaler(inputCol="features",
                            outputCol="scaled_features")
scaled_df = std_scaler.fit(features_df).transform(features_df)
scaled_df.select("scaled_features").show()

+--------------------+
|     scaled_features|
+--------------------+
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
|(4,[0],[7.7375171...|
+--------------------+
only showing top 20 rows



# Feature selection

In [29]:
from pyspark.ml.feature import ChiSqSelector
chisq_selector=ChiSqSelector(numTopFeatures=1,
                             featuresCol="scaled_features",
                             outputCol="selected_features",
                             labelCol="cust_age")
result_df = chisq_selector.fit(scaled_df).transform(scaled_df)
result_df.show(10)

+-----------+----------+--------+------------+----------+------------+--------------------+----+----------+------------+------------+--------+-------------+------+--------------------+--------------------+---------------+-------------+---------------+-------------+-------------+-------------+--------------------+-----------------+
|invoice_num|stock_code|quantity|invoice_date|unit_price|country_code|         description| age|work_class|final_weight|invoice_time|cust_age|working_class|fin_wt|          desc_terms|        desc_nostops|country_indexed|  country_ohe|binarized_price|invoice_month|month_indexed|     features|     scaled_features|selected_features|
+-----------+----------+--------+------------+----------+------------+--------------------+----+----------+------------+------------+--------+-------------+------+--------------------+--------------------+---------------+-------------+---------------+-------------+-------------+-------------+--------------------+-----------------+
|

In [31]:
result_df.show()

+-----------+----------+--------+------------+----------+------------+--------------------+----+----------+------------+------------+--------+-------------+------+--------------------+--------------------+---------------+-------------+---------------+-------------+-------------+-------------+--------------------+-----------------+
|invoice_num|stock_code|quantity|invoice_date|unit_price|country_code|         description| age|work_class|final_weight|invoice_time|cust_age|working_class|fin_wt|          desc_terms|        desc_nostops|country_indexed|  country_ohe|binarized_price|invoice_month|month_indexed|     features|     scaled_features|selected_features|
+-----------+----------+--------+------------+----------+------------+--------------------+----+----------+------------+------------+--------+-------------+------+--------------------+--------------------+---------------+-------------+---------------+-------------+-------------+-------------+--------------------+-----------------+
|

# K-means clustering

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
train_df = result_df.selectExpr("selected_features as features")
kmeans = KMeans(k=3, featuresCol='features')
kmeans_model = kmeans.fit(train_df)
predictions = kmeans_model.transform(train_df)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette measure using squared Euclidean distance = "+ str(silhouette))
cluster_centers = kmeans_model.clusterCenters()
print(cluster_centers)

# Hierarchical clustering using bisecting K-means

In [ ]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
train_df = result_df.selectExpr("selected_features as features")
bkmeans = BisectingKMeans(k=3, featuresCol='features')
bkmeans_model = kmeans.fit(train_df)
predictions = bkmeans_model.transform(train_df)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette measure using squared euclidean distance = "+ str(silhouette))
cluster_centers = kmeans_model.clusterCenters()
print(cluster_centers)

# Topic modeling using latent Dirichlet allocation

In [32]:
from pyspark.ml.clustering import LDA
train_df = result_df.selectExpr("selected_features as features")
lda = LDA(k=3, maxIter=1)
lda_model = lda.fit(train_df)
topics = lda_model.describeTopics(5)
topics.show()
transformed = lda_model.transform(train_df)
transformed.show(5,False)

22/10/22 18:53:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/10/22 18:53:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+-----+-----------+-----------+
|topic|termIndices|termWeights|
+-----+-----------+-----------+
|    0|        [0]|      [1.0]|
|    1|        [0]|      [1.0]|
|    2|        [0]|      [1.0]|
+-----+-----------+-----------+



+-----------+----------+--------+------------+----------+------------+-----------------------------------+----+----------+------------+------------+--------+-------------+------+------------------------------------------+------------------------------------------+---------------+-------------+---------------+-------------+-------------+-------------+---------------------------+-----------------+-----------------------------------------------------------+
|invoice_num|stock_code|quantity|invoice_date|unit_price|country_code|description                        |age |work_class|final_weight|invoice_time|cust_age|working_class|fin_wt|desc_terms                                |desc_nostops                              |country_indexed|country_ohe  |binarized_price|invoice_month|month_indexed|features     |scaled_features            |selected_features|topicDistribution                                          |
+-----------+----------+--------+------------+----------+------------+------------

In [39]:
from pyspark.ml.clustering import GaussianMixture
train_df = result_df.selectExpr("selected_features as features")
gmm = GaussianMixture(k=3, featuresCol='features')
gmm_model = gmm.fit(train_df)
gmm_model.gaussiansDF.show()

+--------------------+--------------------+
|                mean|                 cov|
+--------------------+--------------------+
|[1.1789028430285335]|0.268584721965741...|
|[2.6764305899712513]|0.2162226164941413  |
|[0.1753548046157702]|0.049666706594541...|
+--------------------+--------------------+



# Collaborative filtering using alternating least squares

In [38]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
ratings_df = result_df.selectExpr(
 "CAST(invoice_num AS INT) as user_id",
 "CAST(stock_code AS INT) as item_id",
 "CAST(quantity AS INT) as rating").where("user_id is NOT NULL AND item_id is NOT NULL")
    
ratings_df.show()
(train_df, test_df) = ratings_df.randomSplit([0.7, 0.3])
als = ALS(maxIter=3, regParam=0.03, userCol="user_id",
          itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
als_model = als.fit(train_df)
predictions = als_model.transform(test_df)
evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
user_recs = als_model.recommendForAllUsers(5)
user_recs.show()
item_recs = als_model.recommendForAllItems(5)
item_recs.show()

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
| 489434|  21232|    24|
| 489434|  21523|    10|
| 489434|  21871|    24|
| 489434|  22041|    48|
| 489434|  22064|    24|
| 489434|  85048|    12|
| 489435|  22195|    24|
| 489435|  22349|    12|
| 489435|  22350|    12|
| 489435|  22353|    12|
| 489436|  21181|    12|
| 489436|  21333|     6|
| 489436|  21754|     3|
| 489436|  21755|    18|
| 489436|  21756|     3|
| 489436|  22107|     4|
| 489436|  22109|    16|
| 489436|  22111|    24|
| 489436|  22119|     3|
| 489436|  22142|    12|
+-------+-------+------+
only showing top 20 rows



Root-mean-square error = 91.43290701629488


+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
| 489434|[{22328, 943.4316...|
| 489435|[{84077, 1282.344...|
| 489436|[{35956, 2724.906...|
| 489437|[{22328, 551.1437...|
| 489438|[{16014, 1052.630...|
| 489439|[{35956, 1437.569...|
| 489440|[{84077, 593.5297...|
| 489441|[{22328, 1344.132...|
| 489442|[{84077, 1146.422...|
| 489443|[{21666, 343.8147...|
| 489445|[{21666, 763.3297...|
| 489446|[{35956, 2814.995...|
| 489448|[{84077, 605.7806...|
| 489450|[{84077, 1018.942...|
| 489460|[{21981, 1311.151...|
| 489461|[{21981, 2848.04}...|
| 489464|[{21967, 5594.193...|
| 489465|[{35956, 4722.779...|
| 489488|[{84077, 653.9922...|
| 489505|[{21981, 480.2402...|
+-------+--------------------+
only showing top 20 rows



+-------+--------------------+
|item_id|     recommendations|
+-------+--------------------+
|  10002|[{489539, 142.993...|
|  10080|[{490142, 8.91453...|
|  10109|[{490229, 55.0914...|
|  10120|[{489727, 254.667...|
|  10125|[{489795, 83.9082...|
|  10133|[{489465, 99.0605...|
|  10134|[{489539, 98.5078...|
|  10135|[{537179, 68.7498...|
|  10138|[{489889, 422.959...|
|  11001|[{489723, 150.199...|
|  15034|[{490229, 138.216...|
|  15036|[{537864, 126.045...|
|  15039|[{489671, 45.8432...|
|  16011|[{490142, 167.080...|
|  16012|[{490007, 197.853...|
|  16014|[{536938, 4153.80...|
|  16015|[{489723, 170.436...|
|  16016|[{489723, 172.458...|
|  16046|[{489539, 497.564...|
|  16049|[{536784, 1263.50...|
+-------+--------------------+
only showing top 20 rows

